In [1]:
from Controller import TrainingController, ExplainingController
from Utils.SaveUtils import load_parameters
from Parameters import TrainingParameters, PredictingParameters
import tensorflow as tf
from IPython.core.display import display, HTML
import json
import numpy as np

In [2]:
folder_path = "./SavedModels/%s" % (
"0.8569_BPI2012_BaseLineLSTMModel_2021-05-28 19:02:39.442554" # OW
)   

In [3]:
parameters_json = load_parameters(folder_path=folder_path)
parameters = TrainingParameters(**parameters_json)
tf.random.set_seed(parameters.dataset_split_seed)
np.random.seed(parameters.dataset_split_seed)
parameters.load_model_folder_path = folder_path
predicting_parameters = PredictingParameters()
predicting_parameters.load_model_folder_path = folder_path

In [4]:
trainer = TrainingController(parameters = parameters)


| Running on /job:localhost/replica:0/task:0/device:CPU:0  

| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012/OW 

| Model loaded successfully from: ./SavedModels/0.8569_BPI2012_BaseLineLSTMModel_2021-05-28 19:02:39.442554  


In [5]:
trainer.dataset.df.iloc[[5,5]]

,trace,caseid
5,"[2, 17, 18, 16, 18, 16, 18, 16, 16, 1]",173709
5,"[2, 17, 18, 16, 18, 16, 18, 16, 16, 1]",173709


In [6]:
explainer = ExplainingController(parameters=parameters, predicting_parameters= predicting_parameters)


| Running on /job:localhost/replica:0/task:0/device:CPU:0  

| Model loaded successfully from: ./SavedModels/0.8569_BPI2012_BaseLineLSTMModel_2021-05-28 19:02:39.442554  


In [7]:
########### Get example data from trainer ###########
unbatch_test = trainer.test_dataset.unbatch()
print("Test set length: %d" %(len(list(unbatch_test.as_numpy_iterator()))))
index_from_test = 45

Test set length: 965


In [8]:
example_trace = trainer.dataset.collate_fn([list(unbatch_test.as_numpy_iterator())[index_from_test]])[1][0]

In [9]:
example_trace

array([ 2, 11, 12, 17, 10, 18, 16, 18, 16, 18, 16, 18, 16, 18, 16, 18, 16,
       18, 16, 18, 16, 18, 16, 18, 16, 18, 16, 18,  7,  5,  9, 23, 16, 24,
       22, 24, 22, 24,  4, 22], dtype=int32)

In [10]:
example_trace = trainer.model.vocab.list_of_index_to_vocab(example_trace.tolist())

In [11]:
explainer.show_model_info()

Model: "baseline_lstm_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  928       
_________________________________________________________________
lstm_1 (LSTM)                multiple                  24832     
_________________________________________________________________
sequential_1 (Sequential)    (1, 1, 29)                2141      
Total params: 27,901
Trainable params: 27,773
Non-trainable params: 128
_________________________________________________________________

| Loaded model has been trained for [3050] steps, [50] epochs 


In [12]:
df, data_predicted_list, bn, bn_html, inference, infoBN, markov_blanket_html = explainer.pm_predict_lindaBN_explain(example_trace)


| Selecting Greedy Hill Climbing Algorithm 


In [13]:
generated_html = explainer.generate_html_page_from_graphs("<pre>" +json.dumps(example_trace)+"</pre>", "<pre>" + json.dumps(data_predicted_list) + "</pre>", bn_html, inference, infoBN, markov_blanket_html)

In [14]:
display(HTML(generated_html))